In [1]:
import ee
import geetools

service_account = 'earth-engine-robot@earth-engine-airmove.iam.gserviceaccount.com'
service_file = '/home/dev/airmove/earth-engine/scripts/earth-engine-airmove-596be0e84b06.json'
#service_account = 'ee-drive-uploader@earth-engine-airmove.iam.gserviceaccount.com'
#service_file = '/Users/eric/Documents/jupyter/earth-engine/scripts/earth-engine-airmove-290b51bcebda.json'


credentials = ee.ServiceAccountCredentials(service_account, service_file)
ee.Initialize(credentials)

#ee.Authenticate()
#ee.Initialize()
print("OK")

OK


In [2]:
philippines = ee.Geometry.Polygon(
        [[[116.71687839753203, 18.744929081691417],
          [116.71687839753203, 5.019622515938524],
          [126.91219089753203, 5.019622515938524],
          [126.91219089753203, 18.744929081691417]]], None, False)

ncr = ee.Geometry.Polygon(
        [[[120.86737688113851, 14.816322084944211],
          [120.86737688113851, 14.31656392558974],
          [121.16400774051351, 14.31656392558974],
          [121.16400774051351, 14.816322084944211]]], None, False)

In [3]:
start_date = '2018-06-01'
end_date = '2018-06-03'

era5 = ee.ImageCollection('ECMWF/ERA5/DAILY').select('mean_2m_air_temperature','total_precipitation','u_component_of_wind_10m','v_component_of_wind_10m').filterBounds(philippines).filter(ee.Filter.date(start_date, end_date))
print("OK")


OK


In [4]:
extra = dict(sat='ERA5')

tasks = geetools.batch.Export.imagecollection.toDrive(
            collection=era5,
            folder='airmove-google-earth-engine',
            region=philippines,
            namePattern='{sat}_{system_date}',
            scale=31000,
            datePattern='ddMMMy',
            extra=extra,
            verbose=True,
        )

exporting ERA5_01Jun2018 to folder 'airmove-google-earth-engine' in GDrive
exporting ERA5_02Jun2018 to folder 'airmove-google-earth-engine' in GDrive


In [3]:
##### LC08
import json
ncr_bounds = '/Users/eric/Documents/jupyter/earth-engine/scripts/NCRBoundary.json'

with open(ncr_bounds) as f:
    ncr_shape = json.load(f)
# print(ncr_shape)

ncr_shape = ee.FeatureCollection(ncr_shape)
print(ncr_shape)


ee.FeatureCollection({
  "functionInvocationValue": {
    "functionName": "Collection",
    "arguments": {
      "features": {
        "arrayValue": {
          "values": [
            {
              "functionInvocationValue": {
                "functionName": "Feature",
                "arguments": {
                  "geometry": {
                    "functionInvocationValue": {
                      "functionName": "GeometryConstructors.MultiPolygon",
                      "arguments": {
                        "coordinates": {
                          "constantValue": [
                            [
                              [
                                [
                                  120.98111922531362,
                                  14.498138355398382
                                ],
                                [
                                  120.9813778658114,
                                  14.497491770581458
                                ],
    

In [32]:
ncr_box = ee.Geometry.Polygon([[
    [120.88626682729029, 14.33788811927746],
    [121.15543186635279, 14.33788811927746],
    [121.15543186635279, 14.792458281417941],
    [120.88626682729029, 14.792458281417941],
    [120.88626682729029, 14.33788811927746]]], None, False)

In [72]:

'''
 * Function to mask clouds based on the pixel_qa band of Landsat 8 SR data.
 * @param {ee.Image} image input Landsat 8 SR image
 * @return {ee.Image} cloudmasked Landsat 8 image
 '''

def maskL8sr(image):
    cloud_shadow_bit_mask = (1 << 3)
    clouds_bit_mask = (1 << 5)
    qa = image.select('pixel_qa')
    mask = qa.bitwiseAnd(cloud_shadow_bit_mask).eq(0) and (qa.bitwiseAnd(clouds_bit_mask).eq(0))
    return image.updateMask(mask)

In [74]:
ls08 = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR').filter(ee.Filter.eq('WRS_PATH', 116)).filter(ee.Filter.eq('WRS_ROW', 50)).filterBounds(ncr_shape).filterDate('2017-01-01','2017-02-01').filterMetadata('CLOUD_COVER', 'less_than', 70).map(maskL8sr)


geetools.batch.Export.imagecollection.toDrive(
    collection=ls08,
    folder='earth-engine/try',
    region=ncr_box,
    scale=30,
    namePattern='LS08_{system_date}',
    datePattern='ddMMMy',
    verbose=True
)


exporting LS08_14Jan2017 to folder 'earth-engine/try' in GDrive


[<Task GY24TJTE7ZRLAAFGOPY6VANP EXPORT_IMAGE: LS08_14Jan2017 (UNSUBMITTED)>]

In [75]:
ee.data.getTaskStatus("GY24TJTE7ZRLAAFGOPY6VANP")

[{'state': 'RUNNING',
  'description': 'LS08_14Jan2017',
  'creation_timestamp_ms': 1667723410460,
  'update_timestamp_ms': 1667723425834,
  'start_timestamp_ms': 1667723423410,
  'task_type': 'EXPORT_IMAGE',
  'attempt': 1,
  'id': 'GY24TJTE7ZRLAAFGOPY6VANP',
  'name': 'projects/earthengine-legacy/operations/GY24TJTE7ZRLAAFGOPY6VANP'}]

In [140]:
##### MODIS AODB Daily

collection = ee.ImageCollection('MODIS/006/MCD19A2_GRANULES').select('Optical_Depth_047').filterBounds(ncr_shape)

ncr_bound = ncr_shape.geometry()

startDate = ee.Date.fromYMD(2021, 1, 30)
endDate = ee.Date.fromYMD(2021, 1, 31)
nDays = ee.Number(endDate.difference(startDate, 'day')).round()
daysList = ee.List.sequence(0, nDays)

def getDailyAve(n):
    ini = startDate.advance(n, 'day')
    end = ini.advance(1, 'day')
    datared = collection.filterDate(ini, end).mean()
    day = ini.get("day")
    month = ini.get("month")
    year = ini.get("year")
    #prnt(ee.String(ini.get("day")))
    
    
    print(type(day), type(month), type(year))
    # print(day.int())

    month = ee.String(ini.get("month"))
    year = ee.String(ini.get("year"))
  
    # sysIndexinfo = ee.String("MODIS").cat(ee.String("_")).cat(ee.String(ini.get("day"))).cat(ee.String("_")).cat(ee.String(ini.get("month")))
    # sysIndexinfo = ee.String("MODIS_AODB").cat(ee.String("_")).cat(day).cat(ee.String("_")).cat(month).cat(ee.String("_")).cat(year)
    # sysIndexinfo = ee.String("MODIS_AODB").cat(day).cat("_").cat(month).cat("_").cat(year)
    sysIndexinfo = ee.String("MODIS_AODB")
    return datared.set('year', year).set('month', month).set('day', day).set('system:time_start', ini.millis()).set("system:index", sysIndexinfo)


dailyAve_Listversion = daysList.map(getDailyAve)
dailyAve_Colversion = ee.ImageCollection(dailyAve_Listversion)

print(dailyAve_Colversion)


geetools.batch.Export.imagecollection.toDrive(
    collection=dailyAve_Colversion,
    folder='earth-engine/modis-b',
    region=ncr_box,
    scale=1000,
    verbose=True
)


<class 'ee.ee_number.Number'> <class 'ee.ee_number.Number'> <class 'ee.ee_number.Number'>
<class 'ee.ee_number.Number'> <class 'ee.ee_number.Number'> <class 'ee.ee_number.Number'>


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



exporting 0 to folder 'earth-engine/modis-b' in GDrive
exporting 1 to folder 'earth-engine/modis-b' in GDrive


[<Task LLHBMTD23W7J4WICVDWLMRGF EXPORT_IMAGE: 0 (UNSUBMITTED)>,
 <Task YLXUUMFJXTYBDG64E3NZMDSK EXPORT_IMAGE: 1 (UNSUBMITTED)>]

In [8]:
##### S5P_N02

no2 = ee.ImageCollection('COPERNICUS/S5P/NRTI/L3_NO2').select('NO2_column_number_density').filterBounds(ncr_shape).filterDate('2021-03-01','2021-03-20')
ncr_bound = ncr_shape.geometry()

In [10]:
val_max = 0.0002
val_min = 0
band_viz = {
  "min": val_min,
  "max": val_max, 
  "opacity": 1.0, 
  "palette": ["black","blue","purple","cyan","green","yellow","red"]
}

NCR_mean = no2.mean()

geetools.batch.Export.imagecollection.toDrive(
    collection=no2,
    folder='earth-engine/NO2_GEEExport',
    region=ncr,
    verbose=True
)


exporting 20210301T040730_20210301T045518 to folder 'earth-engine/NO2_GEEExport' in GDrive
exporting 20210301T054730_20210301T063345 to folder 'earth-engine/NO2_GEEExport' in GDrive
exporting 20210302T052824_20210302T061952 to folder 'earth-engine/NO2_GEEExport' in GDrive
exporting 20210303T050824_20210303T055646 to folder 'earth-engine/NO2_GEEExport' in GDrive
exporting 20210304T044830_20210304T053648 to folder 'earth-engine/NO2_GEEExport' in GDrive
exporting 20210305T042830_20210305T051753 to folder 'earth-engine/NO2_GEEExport' in GDrive
exporting 20210305T060830_20210305T065818 to folder 'earth-engine/NO2_GEEExport' in GDrive
exporting 20210306T041330_20210306T050218 to folder 'earth-engine/NO2_GEEExport' in GDrive
exporting 20210306T055330_20210306T064246 to folder 'earth-engine/NO2_GEEExport' in GDrive
exporting 20210307T053330_20210307T061953 to folder 'earth-engine/NO2_GEEExport' in GDrive
exporting 20210308T051330_20210308T060147 to folder 'earth-engine/NO2_GEEExport' in GDrive

[<Task 7CSJMKZZJYAB5VPRDJRTX7OT EXPORT_IMAGE: 20210301T040730_20210301T045518 (UNSUBMITTED)>,
 <Task V6URGT4WIBBBAQMMZ34E6YPT EXPORT_IMAGE: 20210301T054730_20210301T063345 (UNSUBMITTED)>,
 <Task RVDHXBUCLO6MKYYLMGW4BNKX EXPORT_IMAGE: 20210302T052824_20210302T061952 (UNSUBMITTED)>,
 <Task CQ6SHJL7CEUQERW46NZXSQDT EXPORT_IMAGE: 20210303T050824_20210303T055646 (UNSUBMITTED)>,
 <Task SLBKQGITYABX77HIABNIKFSU EXPORT_IMAGE: 20210304T044830_20210304T053648 (UNSUBMITTED)>,
 <Task BTVMMH4OPJ4YTHDWS3FUTPJR EXPORT_IMAGE: 20210305T042830_20210305T051753 (UNSUBMITTED)>,
 <Task GXQDRLNJK7IT5HTWLHER22LO EXPORT_IMAGE: 20210305T060830_20210305T065818 (UNSUBMITTED)>,
 <Task GQU5QYZHKXGOEKJ36GBNEETY EXPORT_IMAGE: 20210306T041330_20210306T050218 (UNSUBMITTED)>,
 <Task AA42IYI5GYK6KYFAIXG5WZVH EXPORT_IMAGE: 20210306T055330_20210306T064246 (UNSUBMITTED)>,
 <Task YVPG335VIEZNWXIBRTK3T2UQ EXPORT_IMAGE: 20210307T053330_20210307T061953 (UNSUBMITTED)>,
 <Task YKQNT4CLQ5A5O2QBQCUMUYYU EXPORT_IMAGE: 20210308T05133